# A. 로딩/점검/기초 EDA (기본 체력)

### A1. 데이터 로딩 및 스키마 리포트

- **문제:** `train.csv`를 읽고, (1) 행/열 수, (2) 각 컬럼 dtype, (3) 결측치 개수/비율을 한 번에 요약한 표를 만드세요.
    
- **무엇을 묻는가:** `read_csv`, `shape`, `dtypes`, `isna().sum()`, 비율 계산, `DataFrame` 구성 능력.
    

### A2. 유니크/중복 점검

- **문제:** PassengerId가 유니크인지 확인하고, 중복이 있다면 중복 행을 반환하세요. 또한 Name 중복 빈도 상위 10개를 뽑으세요.
    
- **무엇을 묻는가:** `duplicated`, `value_counts`, 인덱싱/필터링.
    

### A3. 범주형 분포 빠르게 보기

- **문제:** `Sex`, `Embarked`, `Pclass`의 분포를 “개수 + 비율”로 동시에 보여주는 요약 테이블을 각각 생성하세요.
    
- **무엇을 묻는가:** `value_counts(normalize=...)`, `concat`, 멀티 컬럼 요약 패턴.
    

df_raw     : read_csv 직후, 절대 수정 금지
df         : 분석 대상 (df_raw.copy())
eda_schema : 컬럼 단위 요약 테이블
eda_cat    : 범주형 분포 요약


3️⃣ A단계 실무용 EDA Template (표준 코드)

아래는 Titanic이 아니라도 그대로 쓰는 코드입니다.
---
A0. 데이터 로딩 (Raw 보호)

**왜 필요한가**

- EDA 중 실수로 컬럼을 망가뜨려도 복구 가능

- 실무에서 “원본 훼손”은 신뢰도 문제로 이어짐

In [28]:
import pandas as pd
from pathlib import Path
import os

df_raw = pd.read_csv(
    Path(os.getcwd()).parent
    / "data"
    / "raw"
    / "kaggle"
    / "datasets"
    / "titanic"
    / "titanic"
    / "train.csv"
)
df = df_raw.copy()

A1. 스키마 요약 테이블 (핵심)

**이 테이블 하나로 할 수 있는 판단**

- 결측치 처리 우선순위

- categorical vs numeric 분류

- ID 후보(unique_count == len(df))

- 👉 df.info()는 사람용,
- 👉 이 표는 의사결정용입니다.

In [20]:
eda_schema = pd.DataFrame(
    {
        "dtype": df.dtypes,
        "non_null_count": df.notna().sum(),
        "null_count": df.isna().sum(),
        "null_ratio": df.isna().mean(),
        "unique_count": df.nunique(),
    }
).sort_values("null_ratio", ascending=False)

eda_schema

,dtype,non_null_count,null_count,null_ratio,unique_count
Cabin,object,204,687,0.771044,147
Age,float64,714,177,0.198653,88
Embarked,object,889,2,0.002245,3
PassengerId,int64,891,0,0.000000,891
Name,object,891,0,0.000000,891
Pclass,int64,891,0,0.000000,3
Survived,int64,891,0,0.000000,2
Sex,object,891,0,0.000000,2
Parch,int64,891,0,0.000000,7
SibSp,int64,891,0,0.000000,7


A2. ID / 중복 / 무결성 체크

**실무 해석 가이드**

- ID 유니크 ❌ → 데이터 자체가 신뢰 불가

- duplicated_rows > 0 → 제거가 아니라 원인 분석 대상

In [21]:
integrity_check = {
    "row_count": len(df),
    "passenger_id_unique": df["PassengerId"].is_unique,
    "duplicated_rows": df.duplicated().sum(),
}

integrity_check

{'row_count': 891, 'passenger_id_unique': True, 'duplicated_rows': np.int64(0)}

A3. 범주형 분포 요약 (Reusable Function)
 - 1️⃣ 범주 요약 함수 (템플릿 핵심)
 - 👉 EDA 템플릿에서 가장 재사용률 높은 함수

In [22]:
def summarize_category(series: pd.Series) -> pd.DataFrame:
    cnt = series.value_counts(dropna=False)
    ratio = series.value_counts(dropna=False, normalize=True)
    return pd.DataFrame({"count": cnt, "ratio": ratio})

2️⃣ 적용 예시 (Titanic)

**결과 해석은 이렇게 합니다**

Sex
→ 데이터 불균형 여부

Pclass
→ 사회경제적 편향

Embarked
→ 결측치 의미 여부

In [23]:
eda_sex = summarize_category(df["Sex"])
eda_sex

,count,ratio
Sex,,
male,577,0.647587
female,314,0.352413


In [24]:
eda_pclass = summarize_category(df["Pclass"])
eda_pclass

,count,ratio
Pclass,,
3,491,0.551066
1,216,0.242424
2,184,0.206510


In [25]:
eda_embarked = summarize_category(df["Embarked"])
eda_embarked

,count,ratio
Embarked,,
S,644,0.722783
C,168,0.188552
Q,77,0.086420
NaN,2,0.002245


A4. 숫자형 요약 (EDA의 마무리)

**왜 .T 인가?**

컬럼 중심 사고

이후 eda_schema와 사고 단위가 일치

In [26]:
eda_numeric = df.describe().T
eda_numeric

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292
